In [2]:
import numpy as np
import pandas as pd
import pickle
import xgboost

In [104]:
import lightgbm as lgb
from xgboost.sklearn import XGBClassifier

#warnings.filterwarnings('ignore')
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.metrics import roc_curve, auc, accuracy_score, roc_auc_score, f1_score, confusion_matrix, precision_recall_curve

In [107]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
from collections import Counter, defaultdict
from random import choice


### load data

In [3]:
# sessions
with open ("/Users/nanlin/msds/Adv_ML/leanplum/data/LeanPlum/session.pkl", "rb") as f:
    sessions = pickle.load(f)

# event
with open ("/Users/nanlin/msds/Adv_ML/leanplum/data/LeanPlum/events.pkl", "rb") as f:
    events = pickle.load(f)

# choose certain columns    
session_df = sessions[["user_id_hash", "previous_sessions_duration"]]
event_df = events[["user_id_hash", "event_timestamp", "event", "event_value"]] 

/anaconda3/envs/ml/lib/python3.6/site-packages/ipykernel_launcher.py:3: FutureWarning: 'pandas.core' is private. Use 'pandas.Categorical'
  This is separate from the ipykernel package so we can avoid doing imports until


## label data 

Labels:
We divide the given dataset into training dataset and validation dataset by a cutoff date: Oct 14

In [4]:
# get unique user list 
users = pd.DataFrame(list(events.user_id_hash.unique()))
users.columns = ["user_id_hash"]

#### Use Dec-1 to Dec-7 and Dec-1 to Dec-14 to get labels for the validation set

In [6]:
# first validation label

dec_event = event_df[(event_df["event_timestamp"] >= 1543651199000) & 
                  (event_df["event_timestamp"] < 1544255999000)]

puser1 = set(dec_event[dec_event["event"] == "8"].user_id_hash.unique())
# for validation, the users include all unique users
users_val = users

labels1 = []
for user in list(events.user_id_hash.unique()):
    if user in puser1:
        labels1.append(1)
    else:
        labels1.append(0)
# label and add to val
users_val["purchased1"] = pd.DataFrame(labels1)

In [8]:
# second validation label

dec2_event = event_df[(event_df["event_timestamp"] >= 1544255999000)]
puser2 = set(dec2_event[dec2_event["event"] == "8"].user_id_hash.unique())

labels2 = []
for user in list(events.user_id_hash.unique()):
    if user in puser2:
        labels2.append(1)
    else:
        labels2.append(0)

users_val["purchased2"] = pd.DataFrame(labels2)


#### training labels

In [10]:
# first training label
nov_event = event_df[(event_df["event_timestamp"] >= 1539586800000) &  #Mon Oct 15 2018 00:00:00 GMT-0700
                     (event_df["event_timestamp"] < 1543651199000)] # Fri Nov 30 2018 23:59:59 GMT-0800

puser1 = set(nov_event[nov_event["event"] == "8"].user_id_hash.unique())
users_train = users
labels1 = []

for user in list(events.user_id_hash.unique()):
    if user in puser1:
        labels1.append(1)
    else:
        labels1.append(0)

users_train["purchased1"] = pd.DataFrame(labels1)

In [11]:
# second training label

nov2_event = event_df[(event_df["event_timestamp"] >= 1542268800000) & #Thu Nov 15 2018 00:00:00 GMT-0800
                      (event_df["event_timestamp"] < 1543478400000)] #
puser2 = set(event_df[event_df["event"] == "8"].user_id_hash.unique())
labels2 = []

for user in list(events.user_id_hash.unique()):
    if user in puser2:
        labels2.append(1)
    else:
        labels2.append(0)

users_train["purchased2"] = pd.DataFrame(labels2)

users_train.head()

,user_id_hash,purchased1,purchased2
0,9943447915df3a45fd6720a026af905b6da6b56a37701b...,1,1
1,43f75f8042d3c80c45e222bdd09267f4584684c54d6fae...,0,0
2,999524249720812f2d8c0390293efd58e1ac84d587a01c...,0,0
3,4e6bc35cf7fd79a5312047651e7865915f4a6bec193cf2...,0,0
4,dc009148ee26d658e0240c7b7f6a258790a457737f96e8...,0,0


Feature List:
1. total event value
2. number of purchase
3. whether or not purchase
4. total count of different events in event
5. median, last event time difference to our cutoff time
6. 

In [23]:
nov_e2 = nov_event[nov_event["event"]=="8"][["user_id_hash", "event_value"]].\
                groupby("user_id_hash").sum().reset_index()

In [155]:
event_list = event_df.event.value_counts()[:50].index

In [159]:
def features_event(val_raw,val_user,cut_time):
    
    # feature 1: event value of purchase
    eventvalue = val_raw[val_raw["event"]=="8"][["user_id_hash", "event_value"]].\
                groupby("user_id_hash").sum().reset_index()
    result = val_user.join(eventvalue.set_index("user_id_hash"), on="user_id_hash", how="left")
    
    # feature 2: number of purchase
    purchase_times = val_raw[val_raw["event"]=="8"].groupby("user_id_hash")\
                    .size().reset_index(name='purchase_counts')
    result = result.join(purchase_times.set_index("user_id_hash"), on="user_id_hash", how="left")
    
    # feature 3: avg previous sessions
        df = df.groupby("user_id_hash")["previous_sessions_duration"] \
         .agg('mean').reset_index(name='avg_duration')
    return df
    upre_ = set(nov_event[nov_event['event']=='8'].user_id_hash)
    result['past_buy'] = [1 if uid in user_purd else 0 for uid in result['user_id_hash']]
    
    # feature 4: total count of different events in event: top 50 events
    event_list = ['45', '1', '5', '6', '14', '4', '40', '7', '41', '3', '42',
                  '.a5027911885258752', '.a5400102822346752', '44',
                  '.a5516611293544448', '0', '63', '47', '57', '8', '55',
                  '.m5295687445250048', '.a5061295285075968', '50', '9',
                  '.m6311698772393984', '11', '64', '.a5686782234722304', '54',
                  '.m5295687445250048 Accept', '.a5008055943430144',
                  '.m6311588362059776', '.m5100869650219008',
                  '.m6335456823869440', '.a5933275901722624', '43',
                  '.m5295687445250048 Held Back', '49', '.m5925072799399936',
                  '48', '56', '52', '59', '60', '.m6311698772393984 Accept',
                  '58', '.m5100869650219008 Accept',
                  '.m6311588362059776 Accept', '.m4609541589565440']
        
    for e in event_list:
        event_tmp = val_raw[val_raw["event"]==e].groupby("user_id_hash").size().reset_index(name=f'event{e}_count')
        result = result.join(event_tmp.set_index("user_id_hash"), on="user_id_hash", how="left")
    
    # feature 5: median, last event time difference to cutoff time
    first_last = val_raw.groupby("user_id_hash")['event_timestamp'].agg(['median','last']).reset_index()
    first_last['median_diff'] = cut_time - first_last['median']
    first_last['last_diff'] = cut_time - first_last['last']
    first_last['user_id_hash'] = first_last['user_id_hash'].astype(str)
    result = result.join(first_last[['user_id_hash','median_diff','last_diff']].set_index("user_id_hash"),on="user_id_hash", how="left")
    
    # feature 6: total previous time spent captured by sessions
    
    
    return result

In [ ]:
### Feature for validation/training events

In [ ]:
val_raw = event_df[(event_df["event_timestamp"] >= 1539586800000) & 
                   (event_df["event_timestamp"] < 1543651199000)]
val_feature = features_event(val_raw,users_val,1543651199000)
val_feature = val_feature.apply(lambda x: x.fillna(0))

In [ ]:
train_raw = event_df[(event_df["event_timestamp"] < 1542268800000)]
train_feature = features_event(train_raw,users_train,1542268800000)
train_feature = train_feature.apply(lambda x: x.fillna(0))

In [122]:
train_feature.head()

,user_id_hash,purchased1,purchased2,event_value,purchase_counts,past_buy,event.m6335456823869440 Accept_count,event.m6335456823869440_count,event54_count,event57_count,...,event.a5686782234722304_count,event.m6311698772393984_count,event.a5400102822346752_count,event.a5933275901722624_count,event48_count,event3_count,event64_count,event.m4609541589565440 Accept_count,median_diff,last_diff
0,9943447915df3a45fd6720a026af905b6da6b56a37701b...,1,1,3.492188,1,1,0,1,0,0,...,0,0,1,1,0,0,0,0,2.162291e+09,3.265969e+08
1,43f75f8042d3c80c45e222bdd09267f4584684c54d6fae...,0,0,0.000000,0,0,0,0,0,0,...,0,0,1,0,0,1,0,0,3.815455e+08,3.398837e+08
2,999524249720812f2d8c0390293efd58e1ac84d587a01c...,0,0,0.000000,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,2.438209e+09,2.437954e+09
3,4e6bc35cf7fd79a5312047651e7865915f4a6bec193cf2...,0,0,0.000000,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0.000000e+00,0.000000e+00
4,dc009148ee26d658e0240c7b7f6a258790a457737f96e8...,0,0,0.000000,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,2.045302e+09,1.902918e+09


In [115]:
X_train = train_feature[train_feature.columns.difference(['purchased1','purchased1','user_id_hash'])]
y_train_1 = train_feature[["purchased1"]]
y_train_2 = train_feature[["purchased2"]]

In [116]:
X_val = val_feature[val_feature.columns.difference(["purchased1","purchased2",'user_id_hash'])]
y_val_1 = val_feature[['purchased1']]
y_val_2 = val_feature[['purchased2']]

In [118]:
X_val.head()

,event.a5008055943430144_count,event.a5027911885258752_count,event.a5061295285075968_count,event.a5400102822346752_count,event.a5516611293544448_count,event.a5523274289512448_count,event.a5686782234722304_count,event.a5933275901722624_count,event.m4609541589565440 Accept_count,event.m4609541589565440 Held Back_count,...,event64_count,event6_count,event7_count,event8_count,event9_count,event_value,last_diff,median_diff,past_buy,purchase_counts
0,0,0,0,0,0,0,0,0,0,0,...,0,16,5,1,0,3.492188,1.708996e+09,1.524160e+09,1,1
1,0,1,0,1,0,0,0,0,0,0,...,0,3,2,0,0,0.000000,1.722283e+09,1.763944e+09,0,0
2,0,1,1,1,0,0,0,0,0,0,...,0,0,3,0,0,0.000000,3.820353e+09,3.820608e+09,0,0
3,0,1,0,1,1,0,0,0,0,0,...,0,0,0,0,0,0.000000,1.127654e+09,1.128066e+09,0,0
4,0,1,0,1,1,0,0,0,0,0,...,0,0,0,0,0,0.000000,3.285317e+09,3.427701e+09,0,0


## train with LGB model

In [ ]:
# loop for random search
n_iterations=20

print ("Random search start...")

for col in ['model1_7day','model2_14day']:
    print(f"* training {col} ")
    #y = target[col]
    roc_auc_mean = []
    dict_list = []
    
    for i in range(0, n_iterations):

        param_dist = {'n_estimators' : choice([250,300,350,400,450]),
                  'bagging_fraction': choice([0.5, 0.7, 0.8, 0.9]),
                  'learning_rate': choice([0.05, 0.1, 0.3, 0.5]),
                  'is_unbalance': True,
                  'max_bin': choice([3, 5, 10, 15, 18, 20, 25]),
                  'boosting_type' : choice(['gbdt', 'dart']),
                  'max_depth': choice([2,3,4,5]),      
                  'feature_fraction': choice([0.7, 0.8, 0.9]),
                  'lambda_l1': choice([0, 10, 20, 30, 40]),
                  'objective': 'binary', 
                  'metric': 'auc'} 

        roc_l = []
        
       # y_train_1,y_train_2,X_train
       # y_val_1,y_val_2,X_val

        #for train, test in kf.split(X_tf,y):

        # training
        if col == 'model1_7day':
            y_train = y_train_1
            y_val = y_val_1
        else: 
            y_train = y_train_2
            y_val = y_val_2
            
        gbm = lgb.LGBMClassifier(**param_dist)
        gbm.fit(X_train,y_train)
        # predicting
        y_pred = np.round(gbm.predict_proba(X_val)[:,1],3)
        
        roc = roc_auc_score(y_val, y_pred)
        roc_l.append(roc)

        roc_array = np.asarray(roc_l)

        roc_auc_mean.append(roc_array.mean())
        dict_list.append(param_dist)
        gc.collect()
        

    results_pd = pd.DataFrame({"roc_auc_mean": roc_auc_mean,"parameters": dict_list})    
    results_pd.sort_values("roc_auc_mean", ascending = False, axis = 0, inplace = True)
    
    top_pd = results_pd.head(1)
    
    print(f"--> Best AUC:{top_pd.iloc[0,0]} using {top_pd.iloc[0,1]}")